In [1]:
import numpy as np

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    TextClassificationPipeline,
)
import pandas as pd
from engine.data import prepare_data_for_fine_tuning
from typing import Any

/home/zajkoa/projects/studies/msc_2/nlp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
MODEL_ID = "nghuyong/ernie-2.0-base-en"
MODEL_PATH = "output/liar/ernie/unmasked/1/model_final"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [21]:
test = pd.read_csv("data/liar/valid.csv")
test_dataset = prepare_data_for_fine_tuning(test, tokenizer)

Map:   0%|          | 0/799 [00:00<?, ? examples/s]

Map: 100%|██████████| 799/799 [00:00<00:00, 5269.55 examples/s]


In [22]:
config = AutoConfig.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, config=config)
model.eval()
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=2, device="cuda")

Device set to use cuda


In [23]:
predictions = pipeline(test["text"].tolist())

In [24]:
def convert_prediction(pred: list[dict[str, Any]]) -> np.ndarray:
    if pred[0]["label"] == "LABEL_1":
        return pred[0]["score"]
    else:
        return pred[1]["score"]

In [28]:
((np.array(list(map(convert_prediction, predictions))) >= 0.5) == test.label.values).mean()

0.5344180225281602